#### File to test out Autoencoder and VAE

In [ ]:
#!mkdir data
#!gdown 1CVAQDuPOiwm8h9LJ8a_oOs6zOWS6EgkB
#!gdown 1ykZ9fjTxUwdiEwqagoYZiMcD5aG-7rHe
#!unzip -o test.zip -d data
#!unzip -o train.zip -d data
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/Mamiglia/challenge.git
!wget https://raw.githubusercontent.com/tam4x/aml_challenge/refs/heads/main/preprocess_data.py


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from pathlib import Path
from tqdm import tqdm

from challenge.src.common import load_data, prepare_train_data, generate_submission

#### Create Neural Network Architectures

##### Normal Autoencoder

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TranslatorAE(nn.Module):
    def __init__(self, input_dim, latent_dim, output_dim, hidden_dim=512, n_layers=3, dropout=0.1):
        super().__init__()

        layers = []
        in_dim = input_dim
        for _ in range(n_layers):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dim
        layers.append(nn.Linear(in_dim, latent_dim))
        self.encoder = nn.Sequential(*layers)

        layers = []
        in_dim = latent_dim
        for _ in range(n_layers):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dim
            
        layers.append(nn.Linear(in_dim, output_dim))
        self.decoder = nn.Sequential(*layers)

    def forward(self, x):
        z = self.encoder(x)
        y_pred = self.decoder(z)
        return y_pred


##### Autoencoder with Residual MLP

In [3]:
class ResidualMLPHead(nn.Module):
    def __init__(self, dim, hidden_dim=512, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return x + self.net(x)

class TranslatorRAE(nn.Module):
    def __init__(self, input_dim, latent_dim, output_dim, hidden_dim=512, n_layers=3, dropout=0.1):
        super().__init__()

        layers = []
        in_dim = input_dim
        for _ in range(n_layers):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dim
        layers.append(nn.Linear(in_dim, latent_dim))
        self.encoder = nn.Sequential(*layers)

        layers = []
        in_dim = latent_dim
        for _ in range(n_layers):
            layers.append(nn.Linear(in_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dim
            
        layers.append(nn.Linear(in_dim, output_dim))
        self.decoder = nn.Sequential(*layers)

        self.residual_head = ResidualMLPHead(output_dim, hidden_dim=output_dim//2, dropout=dropout)

    def forward(self, x):
        z = self.encoder(x)
        y_pred = self.decoder(z)
        y_final = self.residual_head(y_pred)
        return y_final


##### VAE

In [4]:
class VAETranslator(nn.Module):
    def __init__(self, input_dim, latent_dim, output_dim, hidden_dims=512, n_layers=3, dropout=0.1):
        super().__init__()

        # Encoder
        layers = []
        in_dim = input_dim
        for _ in range(n_layers):
            layers.append(nn.Linear(in_dim, hidden_dims))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dims
        layers.append(nn.Linear(in_dim, latent_dim * 2))  # output μ and logσ
        self.encoder = nn.Sequential(*layers)

        # Decoder
        layers = []
        in_dim = latent_dim
        for _ in range(n_layers):
            layers.append(nn.Linear(in_dim, hidden_dims))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            in_dim = hidden_dims
        layers.append(nn.Linear(in_dim, output_dim))
        self.decoder = nn.Sequential(*layers)

        # Optional residual refinement
        self.residual_head = ResidualMLPHead(output_dim, hidden_dim=output_dim//2, dropout=dropout)

    def forward(self, x):
        # Encode input → (μ, logσ)
        stats = self.encoder(x)
        mu, log_sigma = stats.chunk(2, dim=-1)
        sigma = torch.exp(log_sigma)

        # Reparameterization trick
        eps = torch.randn_like(sigma)
        z = mu + sigma * eps

        # Decode and refine
        y_base = self.decoder(z)
        y_final = self.residual_head(y_base)
        return y_final, mu, log_sigma

    def kl_loss(self, mu, log_sigma):
        return -0.5 * torch.sum(1 + 2*log_sigma - mu.pow(2) - torch.exp(2*log_sigma), dim=-1).mean()


### Training Loop and NCE Loss aswell as Procrustes Init

In [5]:

class QueueInfoNCELoss(nn.Module):
    def __init__(self, dim, temperature=0.07, queue_size=4096):
        super().__init__()
        self.temperature = temperature
        self.queue_size = queue_size
        # queue shape: (queue_size, dim)
        self.register_buffer("queue", torch.randn(queue_size, dim))
        self.queue = F.normalize(self.queue, dim=1)
        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _enqueue(self, keys):
        """
        keys: tensor (B, dim), already detached, normalized, on same device as queue.
        This writes keys into the circular queue. Safe to call only AFTER backward.
        """
        batch_size = keys.shape[0]
        ptr = int(self.queue_ptr.item())
        end_ptr = (ptr + batch_size) % self.queue_size

        if end_ptr > ptr:
            self.queue[ptr:end_ptr] = keys
        else:
            # wrap
            first_len = self.queue_size - ptr
            self.queue[ptr:] = keys[:first_len]
            self.queue[:end_ptr] = keys[first_len:]
        self.queue_ptr[0] = end_ptr

    def forward(self, z_i, z_j):
        """
        Computes loss using current queue as negatives but does NOT modify the queue.
        z_i: (B, dim) predicted (text -> img)
        z_j: (B, dim) target (image)
        """
        # normalize
        z_i = F.normalize(z_i, dim=1)
        z_j = F.normalize(z_j, dim=1)

        # positive logits: (B, 1)
        l_pos = torch.sum(z_i * z_j, dim=-1, keepdim=True)

        # negative logits from queue: (B, queue_size)
        # queue is a buffer; safe to read
        l_neg = torch.matmul(z_i, self.queue.T)

        # logits: (B, 1 + queue_size)
        logits = torch.cat([l_pos, l_neg], dim=1)
        logits /= self.temperature

        labels = torch.zeros(logits.size(0), dtype=torch.long, device=z_i.device)  # positives at index 0

        loss = F.cross_entropy(logits, labels)
        return loss


In [6]:
# ====== Procrustes initialization ======
def procrustes_init(text_embs, img_embs):
    """
    text_embs: (N, d_text)
    img_embs:  (N, d_img)
    returns: weight matrix (d_img, d_text)
    """
    # Center both
    X = text_embs - text_embs.mean(0, keepdim=True)
    Y = img_embs - img_embs.mean(0, keepdim=True)

    # Compute SVD of cross-covariance
    U, _, Vt = torch.linalg.svd(X.T @ Y, full_matrices=False)
    W = U @ Vt  # orthogonal map d_text→d_img
    return W.T   # shape (d_img, d_text) for nn.Linear weight


def apply_procrustes_init_to_final(model, text_sample, img_sample):
    """
    Apply Procrustes initialization to a model.
    - For MLP / ResidualMLP: apply to final Linear layer (hidden -> img_dim)
    - For TransformerTranslator: apply to first projection (text_dim -> img_dim)
    """
    with torch.no_grad():
        # Compute Procrustes matrix
        W = procrustes_init(text_embs=text_sample, img_embs=img_sample)

        # Apply to the appropriate layer
        applied = False
        for name, m in model.named_modules():
            if isinstance(m, nn.Linear):
                # Transformer: apply to first projection (proj_in)
                if isinstance(model, TranslatorAE) and name.endswith("proj_in"):
                    print(m.weight.shape, W.shape)
                    if m.weight.shape == W.shape:
                        m.weight.copy_(W)
                        applied = True
                        break
                elif isinstance(model, TranslatorRAE) and name.endswith("proj_in"):
                    print(m.weight.shape, W.shape)
                    if m.weight.shape == W.shape:
                        m.weight.copy_(W)
                        applied = True
                        break
                elif isinstance(model, VAETranslator) and name.endswith("proj_in"):
                    print(m.weight.shape, W.shape)
                    if m.weight.shape == W.shape:
                        m.weight.copy_(W)
                        applied = True
                        break
                    
        if not applied:
            print("⚠️ Warning: Could not find matching layer for Procrustes init")
    return model


In [7]:
# ---------- Training loop with Procrustes + InfoNCE ----------
def training(model, train_loader, val_loader, device, epochs, lr, MODEL_PATH,
             use_procrustes_init=True, procrustes_subset=10000, temperature=0.07,
             queue_size=4098):
    """Train LatentSpaceTranslator with optional Procrustes init + InfoNCE loss."""
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-3)
    best_val_loss = float('inf')

    # --- Optional: Procrustes initialization ---
    if use_procrustes_init:
        print("Computing Procrustes initialization...")
        text_list, img_list = [], []
        for i, (X, y) in enumerate(train_loader):
            text_list.append(X.cpu())
            img_list.append(y.cpu())
            if sum(t.shape[0] for t in text_list) >= procrustes_subset:
                break
        text_sample = torch.cat(text_list, dim=0)[:procrustes_subset]
        img_sample = torch.cat(img_list, dim=0)[:procrustes_subset]
        model = apply_procrustes_init_to_final(model, text_sample, img_sample)

    criterion = QueueInfoNCELoss(dim=1536, temperature=temperature, queue_size=queue_size).to(device)
    name = model.__class__.__name__
    # --- Training ---
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            loss = 0.0
            if name == "VAETranslator":
                pred, mu, log_sigma = model(X_batch)
                kl_loss = model.kl_loss(mu, log_sigma)
                loss += 1e-4 * kl_loss
            else:
                pred = model(X_batch)
            # Weighted combination of losses
            loss += 0.3 * criterion(pred, y_batch)
            loss += 1 - F.cosine_similarity(pred, y_batch).mean()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()

            with torch.no_grad():
              keys = F.normalize(y_batch, dim=1).detach()   # image embeddings (targets) as keys
              # put them into the queue
              criterion._enqueue(keys)

        train_loss /= len(train_loader)

        # --- Validation ---
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                loss = 0.0
                if name == "VAETranslator":
                    pred, mu, log_sigma = model(X_batch)
                    kl_loss = model.kl_loss(mu, log_sigma)
                    loss += 1e-4 * kl_loss
                else:
                    pred = model(X_batch)
                # Weighted combination of losses
                loss += 0.3 * criterion(pred, y_batch)
                loss += 1 - F.cosine_similarity(pred, y_batch).mean()

                val_loss += loss.item()

                keys = F.normalize(y_batch, dim=1).detach()   # image embeddings (targets) as keys
                criterion._enqueue(keys)


        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}")

        # --- Save best model ---
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            Path(MODEL_PATH).parent.mkdir(parents=True, exist_ok=True)
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"  ✓ Saved best model (val_loss={val_loss:.6f})")

    return model

### Load the Data

In [9]:
# 4. Data Augmentation
# 5. Zero Shot Stitching
# 6. Triplet Loss / Improve InfoNCE Loss / bidirectional / SimCLR / MoCo
# 7. Autoencoder
# Configuration
EPOCHS = 60
BATCH_SIZE = 512
LR = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load data
#train_data = load_data("drive/MyDrive/data/train/train.npz")
train_data = load_data('data/train/train.npz')
X, y, label = prepare_train_data(train_data)
DATASET_SIZE = len(X)
# Split train/val
# This is done only to measure generalization capabilities, you don't have to
# use a validation set (though we encourage this)
n_train = int(0.9 * len(X))
TRAIN_SPLIT = torch.zeros(len(X), dtype=torch.bool)
TRAIN_SPLIT[:n_train] = 1
X_train, X_val = X[TRAIN_SPLIT], X[~TRAIN_SPLIT]
y_train, y_val = y[TRAIN_SPLIT], y[~TRAIN_SPLIT]

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
y_train.shape, X_train.shape, train_loader.batch_size, val_loader.batch_size

(125000,)
Train data: 125000 captions, 125000 images


(torch.Size([112500, 1536]), torch.Size([112500, 1024]), 512, 512)

### Hyperparameter Optimization

In [10]:
!pip install optuna

In [ ]:
import optuna

def objective_extended(arch, trial, train_dataloader, val_dataloader, device, MODEL_PATH_BASE):

    # --- Common hyperparameters ---
    dropout = trial.suggest_float("dropout", 0.1, 0.3)
    #lr = trial.suggest_loguniform("lr", 5e-4, 1e-2)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)

    # --- New hyperparameters ---
    temperature = trial.suggest_float("temperature", 0.01, 0.2)
    queue_size = trial.suggest_categorical("queue_size", [2048, 4098, 8196])
    #w_infonce = trial.suggest_float("w_infonce", 0.6, 0.8)
    #w_cos = trial.suggest_float("w_cos", 0.4, 1.0)
    #w_mse = trial.suggest_float("w_mse", 1.0 - w_cos, 1.0)
    procrustes_subset = 10000

    # --- Architecture-specific hyperparameters ---
    if arch in ["AE", "RAE", "VAE"]:
        hidden_dim = trial.suggest_categorical("hidden_dim", [512, 1024, 2048])
        num_layers = trial.suggest_int("num_layers", 2, 6)
        if arch == "AE":
            latent_dim = trial.suggest_categorical("latent_dim", [1024, 1536, 2048])
            model = TranslatorAE(
                input_dim=1024, latent_dim=latent_dim, output_dim=1536, hidden_dim=hidden_dim,
                n_layers=num_layers, dropout=dropout
            ).to(device)
        elif arch == "RAE":
            latent_dim = trial.suggest_categorical("latent_dim", [1024, 1536, 2048])
            model = TranslatorRAE(
                input_dim=1024, latent_dim=latent_dim, output_dim=1536, hidden_dim=hidden_dim,
                n_layers=num_layers, dropout=dropout
            ).to(device)
        else:
            latent_dim = trial.suggest_categorical("latent_dim", [512, 768, 1024])
            model = VAETranslator(
                input_dim=1024, latent_dim= latent_dim, output_dim=1536, hidden_dims=hidden_dim,
                n_layers=num_layers, dropout=dropout
            ).to(device)

    # --- Apply Procrustes initialization ---
    # if procrustes_subset > 0:
    #     # Get subset from train_loader
    #     text_list, img_list = [], []
    #     for i, (X, y) in enumerate(train_loader):
    #         text_list.append(X.cpu())
    #         img_list.append(y.cpu())
    #         if sum(t.shape[0] for t in text_list) >= procrustes_subset:
    #             break
    #     text_sample = torch.cat(text_list, dim=0)[:procrustes_subset]
    #     img_sample = torch.cat(img_list, dim=0)[:procrustes_subset]
    #     model = apply_procrustes_init_to_final(model, text_sample, img_sample)

    criterion = QueueInfoNCELoss(dim=1536, temperature=temperature, queue_size=queue_size).to(device)
    # --- Training loop (short run) ---
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6, weight_decay=weight_decay)
    model.train()
    torch.autograd.set_detect_anomaly(True)
    for epoch in range(5):  # short training
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            loss = 0.0
            if arch == "VAE":
                pred, mu, log_sigma = model(X_batch)
                kl_loss = model.kl_loss(mu, log_sigma)
                loss += 1e-4 * kl_loss
            else:
                pred = model(X_batch)
            # Weighted combination of losses
            loss += 0.3 * criterion(pred, y_batch)
            loss += 1 - F.cosine_similarity(pred, y_batch).mean()


            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            with torch.no_grad():
              keys = F.normalize(y_batch, dim=1).detach()   # image embeddings (targets) as keys
              # put them into the queue
              criterion._enqueue(keys)

    # --- Evaluate on validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            loss = 0.0
            if arch == "VAE":
                pred, mu, log_sigma = model(X_batch)
                kl_loss = model.kl_loss(mu, log_sigma)
                loss += 1e-4 * kl_loss
            else:
                pred = model(X_batch)
            # Weighted combination of losses
            loss += 0.3 * criterion(pred, y_batch)
            loss += 1 - F.cosine_similarity(pred, y_batch).mean()
            
            val_loss += loss.item()

            keys = F.normalize(y_batch, dim=1).detach()
            criterion._enqueue(keys)

    val_loss /= len(val_loader)

    return val_loss


def run_optuna_extended(arch, train_dataloader, val_dataloader, device, MODEL_PATH_BASE, n_trials=20):
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective_extended(arch, trial, train_dataloader, val_dataloader, device, MODEL_PATH_BASE),
                   n_trials=n_trials)

    print("Best trial:")
    trial = study.best_trial
    print(f"Val loss: {trial.value}")
    print("Best hyperparameters:")
    for key, value in trial.params.items():
        print(f"  {key}: {value}")

    return trial.params

In [15]:
archs = ['AE', 'RAE', 'VAE']
choosen_arch = archs[2]
best_params = run_optuna_extended(
    arch = choosen_arch,
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    device=DEVICE,
    MODEL_PATH_BASE="models/translator_optuna"
)

[I 2025-11-12 12:16:47,996] A new study created in memory with name: no-name-b1482e2e-09f0-4af3-ab35-aca001611f7a
C:\Users\pier1\AppData\Local\Temp\ipykernel_7940\521694892.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-5, 1e-3)
[W 2025-11-12 12:17:33,013] Trial 0 failed with parameters: {'dropout': 0.364167064448793, 'weight_decay': 2.8136889906024196e-05, 'temperature': 0.14136867285894045, 'queue_size': 8196, 'hidden_dim': 1024, 'num_layers': 3, 'latent_dim': 512} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\pier1\anaconda3\envs\OR\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\pier1\AppData\Local\Temp\ipykernel_7940\521694892.py"

KeyboardInterrupt: 

In [ ]:
if choosen_arch == 'AE':
    model = TranslatorAE(
        input_dim=1024,
        latent_dim=best_params['latent_dim'],
        output_dim=1536,
        hidden_dim=best_params['hidden_dim'],
        n_layers=best_params['n_layers'],
        dropout=best_params['dropout']
    ).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data//models/AE.pth"

elif choosen_arch == 'RAE':
    model = TranslatorRAE(
    input_dim=1024,
    latent_dim=best_params["latent_dim"],
    output_dim=1536,
    hidden_dim=best_params["hidden_dim"],
    n_layers=best_params["num_layers"],
    dropout=best_params["dropout"]).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data/models/RAE.pth"

else:
    model = VAETranslator(
    input_dim=1024,
    latent_dim=best_params["latent_dim"],
    output_dim=1536,
    hidden_dim=best_params["hidden_dim"],
    n_layers=best_params["num_layers"],
    dropout=best_params["dropout"]).to(DEVICE)
    MODEL_PATH = "drive/MyDrive/data/models/VAE.pth"

In [ ]:
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train
print("\n3. Training...")
model = training(model,
                 train_loader,
                 val_loader,
                 DEVICE,
                 EPOCHS,
                 1e-6,
                 MODEL_PATH,
                 True,
                 10000,
                 best_params["temperature"],
                 best_params["queue_size"])

In [ ]:
model.load_state_dict(torch.load(MODEL_PATH))
test_data = load_data("drive/MyDrive/data/test/test.clean.npz")

test_embds = test_data['captions/embeddings']
test_embds = torch.from_numpy(test_embds).float()

with torch.no_grad():
    pred_embds = model(test_embds.to(DEVICE)).cpu()

submission = generate_submission(test_data['captions/ids'], pred_embds, f'{choosen_arch}_submission.csv')
print(f"Model saved to: {MODEL_PATH}")